In [1]:
using PauliPropagation

In [2]:
nq = 8

8

In [3]:
op = PauliString(nq, :Z, round(Int, nq/2))

PauliString(nqubits: 8, 1.0 * IIIZIIII)

In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
m = length(circ)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
opsum = PauliSum(nq, op)

PauliSum(nqubits: 8, 1 Pauli term: 
 1.0 * IIIZIIII
)

In [ ]:
@time dnum = propagate(circ, op, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.591006 seconds (757.89 k allocations: 54.066 MiB, 2.84% gc time, 82.95% compilation time)
length(dnum) = 53247


0.21720058439757214

In [ ]:
@time dnum = propagate(circ, opsum, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.105451 seconds (414 allocations: 3.790 MiB, 6.05% compilation time)
length(dnum) = 53247


0.21720058439757214

In [10]:
dnum

PauliSum(nqubits: 8, 53247 Pauli terms:
 9.3421e-7 * XZIIXXZX
 9.3651e-6 * IIIXIIZY
 3.8189e-6 * YZZIYXZX
 -0.00014415 * ZXYYYXII
 -1.2334e-6 * IZYZIIXY
 2.2079e-7 * XXIIYZXI
 1.4947e-7 * IZZXIZYZ
 3.2705e-7 * YYXIYYYZ
 -0.0035534 * YZIXXZII
 -5.7279e-7 * IXIYIIZX
 -3.3665e-6 * IYZZZYYX
 -8.1232e-9 * YIYZYIZZ
 4.2359e-8 * XIZZXXZZ
 -2.3432e-8 * XZZXZZYZ
 -0.00012064 * ZZXZZXII
 -4.8324e-8 * XZYXIYYZ
 -3.3641e-7 * XXXYIYXZ
 9.2096e-7 * IIIXZZXZ
 5.996e-7 * ZZYXYYYY
 -2.2674e-7 * XXXXZZXY
  ⋮)

#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [11]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [ ]:
wrapped_op = wrapcoefficients(op, NumericPathProperties)
@time dhyb = propagate(circ, wrapped_op, thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
overlapwithzero(dhyb)

  0.418833 seconds (564.80 k allocations: 50.071 MiB, 71.92% compilation time)
length(dhyb) = 53247


0.21720058439757214

In [13]:
dhyb

PauliSum(nqubits: 8, 53247 Pauli terms:
 PathProperty(9.3421e-7) * XZIIXXZX
 PathProperty(9.3651e-6) * IIIXIIZY
 PathProperty(3.8189e-6) * YZZIYXZX
 PathProperty(-0.00014415) * ZXYYYXII
 PathProperty(-1.2334e-6) * IZYZIIXY
 PathProperty(2.2079e-7) * XXIIYZXI
 PathProperty(1.4947e-7) * IZZXIZYZ
 PathProperty(3.2705e-7) * YYXIYYYZ
 PathProperty(-0.0035534) * YZIXXZII
 PathProperty(-5.7279e-7) * IXIYIIZX
 PathProperty(-3.3665e-6) * IYZZZYYX
 PathProperty(-8.1232e-9) * YIYZYIZZ
 PathProperty(4.2359e-8) * XIZZXXZZ
 PathProperty(-2.3432e-8) * XZZXZZYZ
 PathProperty(-0.00012064) * ZZXZZXII
 PathProperty(-4.8324e-8) * XZYXIYYZ
 PathProperty(-3.3641e-7) * XXXYIYXZ
 PathProperty(9.2096e-7) * IIIXZZXZ
 PathProperty(5.996e-7) * ZZYXYYYY
 PathProperty(-2.2674e-7) * XXXXZZXY
  ⋮)

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.
This all looks very messy - and it is! So don't worry about it.

In [ ]:
@time dsym = propagate(circ, op; max_weight=W, max_freq=max_freq);
zerofilter!(dsym);  # Filter the nodes that you find relevant

Converting coefficient of type `Float64` to `NodePathProperties` for the Surrogate.
  1.992429 seconds (18.01 M allocations: 1.035 GiB, 11.90% gc time, 25.00% compilation time)


In [15]:
@time evaluate!(dsym, thetas)
overlapwithzero(dsym)

  0.114103 seconds (77.83 k allocations: 5.424 MiB, 515.99% compilation time)


0.21720058439757214